In [1]:
import urllib.request, json 
import pandas as pd
import numpy as np
import math

In [2]:
data = pd.read_csv("clean_csv/akoya.csv")

managers = ["Ali","Ruslan","Sami","Yahya","Youssef","Santi","Shrey","Dani"]

current_gw = 36 #current gameweek

In [3]:
data.head()

,player_id,player_name,position,team,gameweek,manager_id,squad_position,bench,points,minutes,...,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,total_points,in_dreamteam
0,1.0,Cédric,DEF,FUL,5.0,transfer market,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1,2.0,Leno,GK,FUL,5.0,transfer market,NaN,NaN,2.0,90.0,...,0.0,2.0,0.0,13.0,19.0,10.0,0.0,2.9,2.0,False
2,105.0,Duffy,DEF,FUL,5.0,transfer market,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,200.0,Cairney,MID,FUL,5.0,transfer market,NaN,NaN,1.0,12.0,...,0.0,0.0,0.0,4.0,0.0,1.7,4.0,0.6,1.0,False
4,201.0,Ream,DEF,FUL,5.0,transfer market,NaN,NaN,2.0,90.0,...,0.0,0.0,0.0,17.0,16.2,0.7,0.0,1.7,2.0,False


# Points

In [4]:
gameweek_points = data[(data["manager_id"]!="transfer market")&(data["bench"]=="fielded")]

grouped = gameweek_points.groupby(["manager_id","gameweek"])["points"].sum()
gameweek_df = grouped.reset_index()
gameweek_df

,manager_id,gameweek,points
0,Ali,5.0,51.0
1,Ali,6.0,43.0
2,Ali,7.0,0.0
3,Ali,8.0,54.0
4,Ali,9.0,53.0
...,...,...,...
243,Youssef,31.0,36.0
244,Youssef,32.0,27.0
245,Youssef,33.0,16.0
246,Youssef,34.0,57.0


In [5]:
def get_ranking(df,bench,goal):
    filtered = df[(df["manager_id"]!="transfer market")&(df["bench"]==bench)&(df["gameweek"]!=7)]
    ranked_df = filtered.groupby(df["manager_id"])
    return ranked_df[goal].sum().sort_values(ascending=False)

In [6]:
get_ranking(data,"fielded","points")

manager_id
Ali        1418.0
Ruslan     1339.0
Sami       1314.0
Yahya      1285.0
Youssef    1264.0
Santi      1150.0
Shrey      1149.0
Dani       1082.0
Name: points, dtype: float64

### League by positions

Make 4 tables, ranking everyone depending on each position (gk, def, mid, fwd)

In [7]:
#GK Table
gk = data[data["position"]=="GK"]
get_ranking(gk,"fielded","points")

manager_id
Shrey      143.0
Ruslan     133.0
Ali        122.0
Dani       120.0
Sami       119.0
Santi      116.0
Youssef    116.0
Yahya       98.0
Name: points, dtype: float64

In [8]:
#DEF Table
deef = data[data["position"]=="DEF"]
get_ranking(deef,"fielded","points")

manager_id
Sami       477.0
Ruslan     422.0
Youssef    382.0
Dani       370.0
Ali        360.0
Yahya      329.0
Santi      300.0
Shrey      292.0
Name: points, dtype: float64

In [9]:
#MID Table
mid = data[data["position"]=="MID"]
get_ranking(mid,"fielded","points")

manager_id
Youssef    601.0
Yahya      559.0
Ali        555.0
Shrey      549.0
Ruslan     513.0
Sami       465.0
Santi      443.0
Dani       407.0
Name: points, dtype: float64

In [10]:
#FWD Table
fwd = data[data["position"]=="FWD"]
get_ranking(fwd,"fielded","points")

manager_id
Ali        381.0
Yahya      299.0
Santi      291.0
Ruslan     271.0
Sami       253.0
Dani       185.0
Shrey      165.0
Youssef    165.0
Name: points, dtype: float64

### Best gw by position

In [11]:
def get_ranking_gw(df):
    filtered = df[(df["manager_id"]!="transfer market")&(df["bench"]=="fielded")&(df["gameweek"]!=7)]
    ranked_df = filtered.groupby(["manager_id","gameweek"])
    return ranked_df["points"].sum().sort_values(ascending=False)

In [12]:
#GK GW Table
get_ranking_gw(gk)

manager_id  gameweek
Sami        34.0        15.0
Shrey       12.0        15.0
Sami        11.0        14.0
Ali         29.0        14.0
Shrey       11.0        14.0
                        ... 
Sami        28.0         0.0
Dani        30.0         0.0
Youssef     26.0        -1.0
Sami        35.0        -2.0
Ali         24.0        -3.0
Name: points, Length: 240, dtype: float64

In [13]:
#DEF GW Table
get_ranking_gw(deef)

manager_id  gameweek
Ruslan      25.0        47.0
            27.0        41.0
Youssef     25.0        36.0
Sami        29.0        35.0
Youssef     34.0        29.0
                        ... 
Ruslan      18.0         2.0
Santi       9.0          1.0
Shrey       28.0         1.0
            32.0         0.0
Santi       8.0         -3.0
Name: points, Length: 240, dtype: float64

In [14]:
#MID GW Table
get_ranking_gw(mid)

manager_id  gameweek
Ali         29.0        44.0
Shrey       20.0        42.0
            9.0         41.0
Youssef     23.0        40.0
            14.0        40.0
                        ... 
Dani        32.0         4.0
Santi       12.0         4.0
Ali         32.0         3.0
            28.0         2.0
Dani        25.0         2.0
Name: points, Length: 240, dtype: float64

In [15]:
#FWD GW Table
get_ranking_gw(fwd)

manager_id  gameweek
Santi       34.0        28.0
Ali         9.0         25.0
            5.0         24.0
            8.0         24.0
Ruslan      29.0        23.0
                        ... 
Sami        6.0          1.0
Youssef     13.0         1.0
Ruslan      19.0         0.0
Dani        25.0         0.0
Santi       8.0          0.0
Name: points, Length: 240, dtype: float64

### Bench FC


In [16]:
get_ranking(data,"bench","points")

manager_id
Ruslan     241.0
Sami       236.0
Santi      232.0
Dani       207.0
Ali        205.0
Shrey      157.0
Youssef    154.0
Yahya      144.0
Name: points, dtype: float64

### Optimised Bench

In [17]:
def optimise(gw,manager):
    benched = data[(data["gameweek"] == gw)&(data["manager_id"] == manager)&(data["bench"] == "bench")]
    fielded = data[(data["gameweek"] == gw)&(data["manager_id"] == manager)&(data["bench"] == "fielded")]

    points = 0

    positions = [("DEF",3),("MID",2),("FWD",1)]

    b_gk = benched[benched["position"] == "GK"]
    f_gk = fielded[fielded["position"] == "GK"]

    if  f_gk.loc[f_gk.index.tolist()[0],"points"] < b_gk.loc[b_gk.index.tolist()[0],"points"]:
        points += b_gk["points"].item() - f_gk["points"].item()
        benched = benched.drop(index=b_gk.index)

    for indx, b_player in benched.iterrows():
        for pos in positions:
            for indx2, f_player in fielded.iterrows():
                if len(fielded[fielded["position"] == pos[0]]) > pos[1]:
                    if f_player["points"] < b_player["points"]:
                        points += b_player["points"] - f_player["points"]
                        benched = benched.drop(index=indx)
                        fielded = fielded.drop(index=indx2)
                        break  # exit the inner loop and go to the next b_player
            else:
                continue  # only executed if the inner loop didn't break
            break  # exit the outer loop and go to the next b_player

    return points

In [18]:
missed_points = pd.DataFrame(columns=["manager","gameweek","missed_pts"])

for manager in managers:
    for gw in range(5,current_gw):
        new_row = {"manager":manager,"gameweek":gw,"missed_pts":optimise(gw,manager)}
        missed_points = missed_points.append(new_row, ignore_index=True)


C:\Users\santi\AppData\Local\Temp\ipykernel_32124\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_32124\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_32124\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_32124\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

In [19]:
missed_points

,manager,gameweek,missed_pts
0,Ali,5,1.0
1,Ali,6,5.0
2,Ali,7,0.0
3,Ali,8,2.0
4,Ali,9,0.0
...,...,...,...
243,Dani,31,1.0
244,Dani,32,1.0
245,Dani,33,0.0
246,Dani,34,0.0


In [20]:
missed_points_grouped = missed_points.groupby(missed_points["manager"])
missed_points_grouped["missed_pts"].sum().sort_values(ascending=False)

manager
Sami       155.0
Ruslan     134.0
Santi      132.0
Ali        130.0
Dani       110.0
Shrey      101.0
Youssef     96.0
Yahya       82.0
Name: missed_pts, dtype: float64

### Trade Winner & Loser TBD


### GW Podiums

In [21]:
gw_podiums = pd.DataFrame(columns=["manager","gameweek","points"])

gw_podiums["manager"] = gameweek_df["manager_id"]
gw_podiums["gameweek"] = gameweek_df["gameweek"]
gw_podiums["points"] = gameweek_df["points"]
gw_podiums["rank"] = gw_podiums.groupby("gameweek")["points"].rank(ascending=False).apply(math.floor)
gw_podiums["podium"] = gw_podiums["rank"]<4


gw_podiums

,manager,gameweek,points,rank,podium
0,Ali,5.0,51.0,1,True
1,Ali,6.0,43.0,2,True
2,Ali,7.0,0.0,4,False
3,Ali,8.0,54.0,1,True
4,Ali,9.0,53.0,2,True
...,...,...,...,...,...
243,Youssef,31.0,36.0,7,False
244,Youssef,32.0,27.0,4,False
245,Youssef,33.0,16.0,8,False
246,Youssef,34.0,57.0,4,False


In [22]:
gw_podiums[gw_podiums["manager"] == "Youssef"]

,manager,gameweek,points,rank,podium
217,Youssef,5.0,36.0,5,False
218,Youssef,6.0,33.0,6,False
219,Youssef,7.0,0.0,4,False
220,Youssef,8.0,27.0,6,False
221,Youssef,9.0,31.0,6,False
222,Youssef,10.0,48.0,3,True
223,Youssef,11.0,48.0,4,False
224,Youssef,12.0,40.0,3,True
225,Youssef,13.0,33.0,5,False
226,Youssef,14.0,61.0,2,True


In [23]:
total_podiums_df = pd.DataFrame(index=['1st', '2nd', '3rd'], columns=gw_podiums['manager'].unique())

grouped = gw_podiums.groupby(['manager', 'rank']).size()

# loop through each manager and count their 1st, 2nd, and 3rd place finishes
for manager in total_podiums_df.columns:
    try:
        total_podiums_df.loc['1st', manager] = grouped[manager, 1]
    except KeyError:
        total_podiums_df.loc['1st', manager] = 0
    try:
        total_podiums_df.loc['2nd', manager] = grouped[manager, 2]
    except KeyError:
        total_podiums_df.loc['2nd', manager] = 0
    try:
        total_podiums_df.loc['3rd', manager] = grouped[manager, 3]
    except KeyError:
        total_podiums_df.loc['3rd', manager] = 0

total_podiums_df.loc['Total'] = total_podiums_df.sum(axis=0)

total_podiums_df

,Ali,Dani,Ruslan,Sami,Santi,Shrey,Yahya,Youssef
1st,8,2,7,6,2,2,3,0
2nd,5,2,2,2,3,4,7,6
3rd,4,3,5,4,5,2,3,6
Total,17,7,14,12,10,8,13,12


### Tottenham Award

Most GW without podium

In [24]:
def longest_streak(df,who):
    current_streak = 1
    max_streak = 1
    indx = 0
    
    df = df[df["manager"]==who]
    
    series = df["podium"]

    for i in range(1, len(series)):
        if series.iloc[i]:
            max_streak = max(max_streak, current_streak)
            current_streak = 1
        else:
            current_streak += 1
            if current_streak > max_streak:
                indx = i

    return max(max_streak, current_streak),df.iloc[indx]["manager"]

In [25]:
longest_nopodium_streak = (0,"")

for manager in managers:
    streak = longest_streak(gw_podiums, manager)
    if longest_nopodium_streak[0] < streak[0]:
        longest_nopodium_streak = streak

longest_nopodium_streak

(14, 'Dani')

### GW Losers

In [26]:
last_df = pd.DataFrame(index=["Last"], columns=gw_podiums['manager'].unique())

last_grouped = gw_podiums.groupby(['manager', 'rank']).size()

# loop through each manager and count their 1st, 2nd, and 3rd place finishes
for manager in last_df.columns:
    try:
        last_df.loc['Last', manager] = last_grouped[manager, 8]
    except KeyError:
        last_df.loc['1st', manager] = 0


last_df

,Ali,Dani,Ruslan,Sami,Santi,Shrey,Yahya,Youssef
Last,2,7,4,2,3,6,3,2


# Players

### 1-team Players

In [27]:
def get_loyalty(data):
    grouped = data.groupby(['manager_id',"player_name","team"]).size().sort_values(axis=0,ascending=False)
    loyalty = grouped.to_frame().sort_values(by=[0, 'manager_id'], ascending=[False, True])
    return loyalty

In [28]:
#most owned players per club
no_tm = data[(data["manager_id"]!="transfer market")]
loyalty = get_loyalty(no_tm)
loyalty

0
manager_id player_name team    
Ali        Haaland     MCI   31
           Rashford    MUN   31
Dani       Aké         MCI   31
           Sánchez     BHA   31
           Mbeumo      BRE   31
...                          ..
Shrey      Sancho      MUN    1
Yahya      Chilwell    CHE    1
Youssef    Bamford     LEE    1
           Bowen       WHU    1
           Sinisterra  LEE    1

[429 rows x 1 columns]

In [29]:
loyalty.loc["Shrey"]

,,0
player_name,team,
Fernandes,MUN,31
Stones,MCI,31
Alisson,LIV,31
Lloris,TOT,31
Maddison,LEI,30
Foden,MCI,27
Mahrez,MCI,26
Son,TOT,24
Vardy,LEI,19


In [30]:
#most fielded players per club
most_played = get_loyalty(gameweek_points)
most_played

0
manager_id player_name team    
Yahya      Kane        TOT   31
Youssef    Martinelli  ARS   31
Shrey      Fernandes   MUN   30
           Alisson     LIV   30
Yahya      Saka        ARS   30
...                          ..
Youssef    Ings        WHU    1
           Jorginho    ARS    1
           Bowen       WHU    1
           Henderson   NFO    1
           Antonio     WHU    1

[394 rows x 1 columns]

### Globe Trotters

Players in the most teams

In [31]:
most_teams_grouped = loyalty.groupby(["player_name","team"]).size().sort_values(axis=0,ascending=False)
most_teams_df = most_teams_grouped.to_frame().sort_values(by=[0, 'player_name'], ascending=[False, True])
most_teams_df[most_teams_df.iloc[:,0]>3]

,,0
player_name,team,
Chilwell,CHE,5
Sancho,MUN,5
Edouard,CRY,4
Eriksen,MUN,4
Gündogan,MCI,4
Martial,MUN,4
Martínez,MUN,4
Walker,MCI,4


### Club Mascot

In [32]:
def get_clubmascot(data):
    mascot_grouped = data.groupby(['manager_id',"team"]).sum().sort_values(by=[0, 'manager_id'], ascending=[False, True])
    return mascot_grouped

In [42]:
#most owned club
club_mascot = get_clubmascot(most_played)
club_mascot["bench"] = get_clubmascot(loyalty)
club_mascot.rename(columns={0: 'fielded'}, inplace=True)
club_mascot["ratio"] = round(club_mascot["fielded"]/club_mascot["bench"],2)
club_mascot

fielded  bench  ratio
manager_id team                       
Ruslan     ARS       184    224   0.82
Ali        MCI        89    111   0.80
Dani       BRE        84     98   0.86
Sami       LIV        65     77   0.84
Youssef    MCI        64     83   0.77
...                  ...    ...    ...
Dani       LEE         1      3   0.33
           WHU         1      1   1.00
Ruslan     EVE         1      1   1.00
           LEI         1      3   0.33
Sami       NFO         1      1   1.00

[130 rows x 3 columns]

In [43]:
club_mascot.loc["Santi"]

,fielded,bench,ratio
team,,,
MUN,45,67,0.67
LIV,42,52,0.81
CHE,36,56,0.64
AVL,32,35,0.91
NEW,27,40,0.68
TOT,23,26,0.88
BHA,22,31,0.71
MCI,20,29,0.69
LEE,18,19,0.95


# Stats

### Most Goals

In [45]:
get_ranking(data,"fielded","goals_scored")

manager_id
Ali        74.0
Ruslan     62.0
Yahya      62.0
Santi      55.0
Sami       49.0
Youssef    48.0
Shrey      41.0
Dani       33.0
Name: goals_scored, dtype: float64

In [ ]:
get_ranking(data,"bench","goals_scored")

manager_id
Ruslan     7.0
Ali        6.0
Dani       6.0
Santi      6.0
Shrey      5.0
Youssef    5.0
Yahya      2.0
Sami       1.0
Name: goals_scored, dtype: float64

### Most Assists

In [ ]:
get_ranking(data,"fielded","assists")

manager_id
Ali        47.0
Shrey      46.0
Yahya      45.0
Youssef    44.0
Ruslan     43.0
Sami       42.0
Dani       41.0
Santi      23.0
Name: assists, dtype: float64

In [ ]:
get_ranking(data,"bench","assists")

manager_id
Sami       6.0
Ruslan     4.0
Youssef    4.0
Ali        3.0
Dani       3.0
Shrey      3.0
Santi      2.0
Yahya      0.0
Name: assists, dtype: float64

### Most Clean Sheets

In [ ]:
gk_def = data[(data["position"]=="GK")|(data["position"]=="DEF")]

get_ranking(gk_def,"fielded","clean_sheets")

manager_id
Sami       51.0
Ruslan     49.0
Ali        46.0
Youssef    46.0
Dani       40.0
Yahya      40.0
Santi      39.0
Shrey      35.0
Name: clean_sheets, dtype: float64

In [ ]:
get_ranking(gk_def,"bench","clean_sheets")

manager_id
Ruslan     14.0
Sami       13.0
Santi      13.0
Dani       10.0
Ali         9.0
Yahya       9.0
Youssef     9.0
Shrey       8.0
Name: clean_sheets, dtype: float64

### Avg Minutes Played

In [ ]:
round(get_ranking(data,"fielded","minutes")/11/31)

manager_id
Ali        81.0
Sami       79.0
Ruslan     79.0
Yahya      78.0
Youssef    78.0
Dani       76.0
Santi      76.0
Shrey      70.0
Name: minutes, dtype: float64

### Maguire Award
Most Goals Conceded

In [ ]:
get_ranking(gk_def,"fielded","goals_conceded")

manager_id
Ruslan     182.0
Youssef    177.0
Ali        171.0
Dani       171.0
Sami       168.0
Santi      153.0
Shrey      149.0
Yahya      137.0
Name: goals_conceded, dtype: float64

### 2016 Pessi Award
Most Missed Pens

In [ ]:
get_ranking(data,"fielded","penalties_missed")

manager_id
Dani       4.0
Sami       2.0
Santi      2.0
Yahya      2.0
Ruslan     1.0
Shrey      1.0
Youssef    1.0
Ali        0.0
Name: penalties_missed, dtype: float64

### Not De Gea Award
Most Pens Saved

In [ ]:
get_ranking(data,"fielded","penalties_saved")

manager_id
Sami       2.0
Ali        1.0
Dani       1.0
Shrey      1.0
Ruslan     0.0
Santi      0.0
Yahya      0.0
Youssef    0.0
Name: penalties_saved, dtype: float64

### Prime Maguire Award
Most Own Goals

In [ ]:
get_ranking(data,"fielded","own_goals")

manager_id
Santi      3.0
Dani       2.0
Ali        1.0
Sami       1.0
Shrey      1.0
Youssef    1.0
Ruslan     0.0
Yahya      0.0
Name: own_goals, dtype: float64

### Sergio Ramos Award
Most Red Cards

In [ ]:
get_ranking(data,"fielded","red_cards")

manager_id
Ruslan     2.0
Santi      2.0
Sami       1.0
Ali        0.0
Dani       0.0
Shrey      0.0
Yahya      0.0
Youssef    0.0
Name: red_cards, dtype: float64

### Sergio Ramos Lite Award
Most Yellow Cards

In [ ]:
get_ranking(data,"fielded","yellow_cards")

manager_id
Dani       50.0
Ali        46.0
Yahya      46.0
Sami       40.0
Santi      33.0
Shrey      33.0
Ruslan     32.0
Youssef    32.0
Name: yellow_cards, dtype: float64

### The Wall Award
Most Saves

In [ ]:
get_ranking(data,"fielded","saves")

manager_id
Dani       114.0
Sami       104.0
Shrey       99.0
Ruslan      94.0
Youssef     80.0
Santi       79.0
Ali         75.0
Yahya       39.0
Name: saves, dtype: float64

### BPS Merchant

In [ ]:
filtered_bps = data[(data["manager_id"]!="transfer market")&(data["bench"]=="bench")]
df = filtered_bps.groupby(data["manager_id"])
df["bonus"].sum().sort_values(ascending=False)

manager_id
Sami       30.0
Santi      29.0
Shrey      24.0
Dani       20.0
Ruslan     16.0
Yahya      16.0
Ali        15.0
Youssef    13.0
Name: bonus, dtype: float64

### TOTW Merchant

In [ ]:
get_ranking(data,"fielded","in_dreamteam")

manager_id
Ali        34
Sami       28
Yahya      26
Ruslan     24
Youssef    18
Shrey      17
Santi      11
Dani       10
Name: in_dreamteam, dtype: object